### Configuration

In [2]:
namespace = "ci-ska-mid-psi-1459164122-jaredmda"
vccs = [1, 2]
fsps = [1, 2, 3, 4]

### Setup

In [3]:
import os

import notebook_tools.histogram_client as HistogramClient
from notebook_tools.misc_helper import get_tango_host
from PyTango import DeviceProxy


os.environ["TANGO_HOST"] = get_tango_host(namespace)

# Set to the talon board you want to check

In [3]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_histogram(device, data):
    axis_range = np.linspace(-1, 1, 64)
    fig = make_subplots(
        rows=2,
        cols=4,
        shared_xaxes=True,
        shared_yaxes=True,
        column_titles=("Pol-X", "Pol-X", "Pol-Y", "Pol-Y"),
        column_widths=[4, 1, 4, 1],
        row_heights=[4, 1],
        x_title="Real",
        y_title="Imaginary",
    )

    for idx, pol in {0: "X", 1: "Y"}.items():
        # histogram contour of samples per polarization
        pol_data = data[idx]
        fig.add_trace(
            go.Contour(
                x=axis_range,
                y=axis_range,
                z=pol_data,
                name=f"Pol-{pol}",
            ),
            row=1,
            col=1 + idx * 2,
        )

        # sum all rows for real component
        real_counts = np.sum(pol_data, axis=0)
        fig.add_trace(
            go.Bar(
                x=axis_range,
                y=real_counts,
            ),
            row=2,
            col=1 + idx * 2,
        )

        # sum all columns for imaginary component
        imag_counts = np.sum(pol_data, axis=1)
        fig.add_trace(
            go.Bar(
                y=axis_range,
                x=imag_counts,
                orientation="h",
            ),
            row=1,
            col=2 + idx * 2,
        )

        # calculate power in dB
        samples = real_counts.sum()
        if samples > 0:
            power = (
                np.sum(
                    np.asarray(pol_data)
                    * np.power(
                        np.abs(axis_range + 1j * axis_range),
                        2,
                    )
                )
                / samples
            )
            power_db = 20 * np.log10(power)
        else:
            power_db = np.NaN
        print(f"{device} pol-{pol}: {samples} samples, power = {power_db:.1f} dB")

    width = 1024
    fig.update_layout(
        title=f"Histograms of complex signals from {device}, channel 0",  # TODO add support for channel selection, default is 0
        width=width,
        height=width // 2,
        showlegend=False,
    )

    fig.show()

### Wideband State Count

In [4]:
import math

import plotly.graph_objects as go
from plotly.subplots import make_subplots


def read_vectors(dp):
    state_count_vec_1 = dp.read_attribute("state_count_vector_1").value
    state_count_vec_2 = dp.read_attribute("state_count_vector_2").value
    power_spectrum_1 = dp.read_attribute("psd_vector_1").value
    power_spectrum_2 = dp.read_attribute("psd_vector_2").value

    # state count vector and psd vector 3/4 are not used for band 1/2
    dp.command_inout("state_count_reset")
    return state_count_vec_1, state_count_vec_2, power_spectrum_1, power_spectrum_2


def plot_wb_state_count(
    device,
    n_bins,
    state_count_vec_1,
    state_count_vec_2,
    power_spectrum_1,
    power_spectrum_2,
):
    x_axis = list(range(-math.ceil(n_bins / 2), math.floor(n_bins / 2) + 1))
    psd_x = list(range(1, len(power_spectrum_1) + 1))
    normalized_x_axis = [
        (x - min(x_axis)) / (max(x_axis) - min(x_axis)) * (2.0) - 1.0 for x in x_axis
    ]
    normalized_psd_x = [(x - min(psd_x)) / (max(psd_x) - min(psd_x)) * 1.0 for x in psd_x]

    fig = make_subplots(
        rows=2,
        cols=2,
        vertical_spacing=0.2,
        subplot_titles=(
            "State Count (pol X/A sub-band 1)",
            "State Count (pol Y/B sub-band 1)",
            "Power Spectrum (Pol X/A sub-band 1)",
            "Power Spectrum (pol Y/B sub-band 1)",
        ),
    )

    fig["layout"]["plot_bgcolor"] = "#fafafa"
    fig["layout"]["showlegend"] = False
    fig.add_trace(
        go.Bar(x=normalized_x_axis, y=state_count_vec_1, name="state count 1"), row=1, col=1
    )
    fig.add_trace(
        go.Bar(x=normalized_x_axis, y=state_count_vec_2, name="state count 2"), row=1, col=2
    )
    fig.add_trace(
        go.Bar(x=normalized_psd_x, y=power_spectrum_1, name="power spectrum 1"), row=2, col=1
    )
    fig.add_trace(
        go.Bar(x=normalized_psd_x, y=power_spectrum_2, name="power spectrum 2"), row=2, col=2
    )
    fig.update_layout(title_text=f"{device} sub-band 1", height=700)
    fig.show()

In [ ]:
for vcc in vccs:
    device = f"talondx-00{vcc}/wbstatecount/state_count"
    n_bins = 4096
    sample_accum = 100000

    dp = DeviceProxy(device)
    dp.command_inout("state_count_reset")

    dp.write_attribute("sample_accum", sample_accum)
    dp.write_attribute("bin_number_select", n_bins)
    dp.command_inout("state_count_capture")
    state_count_vec_1, state_count_vec_2, power_spectrum_1, power_spectrum_2 = read_vectors(dp)
    plot_wb_state_count(
        device,
        n_bins,
        state_count_vec_1,
        state_count_vec_2,
        power_spectrum_1,
        power_spectrum_2,
    )

### Wideband Input Buffer 

In [ ]:
for vcc in vccs:
    device = f"talondx-00{vcc}/wideband-input-buffer/wideband-input-buffer"
    dp = DeviceProxy(device)

    receptor_id = dp.read_attribute("DishID").value
    rx_sample_rate = dp.read_attribute("RxSampleRate").value

    meta_transport_sample_rate = dp.read_attribute("MetaTransportSampleRate").value

    print(f"Receptor ID = {receptor_id}")

    # the number of sample pairs received per PPS interval
    print(f"Rx Sample Rate (sample pairs/pps interval) = {rx_sample_rate}")

    # The sample rate in samples per second
    print(f"Meta Transport Sample Rate (samples/s) = {meta_transport_sample_rate}")
    print("------------------------------------------------------------------")

### Pre-VCC Histogram

In [ ]:
for vcc in vccs:
    device = f"talondx-00{vcc}/histogram/e_pre_vcc"
    timeout = 3000  # ms
    client = HistogramClient(device, timeout)

    # data is an array of [pol_x_data, pol_y_data], or None if success is false.
    success, data = client.capture()
    print(f"Histogram capture successful: {success}")

    if success:
        plot_histogram(device, data)

### Packet Stream Repair

In [ ]:
# Each board has 4 packet stream repair devices packet_stream_repair_[0-3]
from time import sleep

for vcc in vccs:
    for fsp in fsps:
        device = f"talondx-00{fsp}/packetstreamrepair/packet_stream_repair_{vcc-1}"
        dp = DeviceProxy(device)

        packet_rate = dp.read_attribute("packet_rate").value
        # Read the number of packets received per second.
        # Wait 1 second to read the rx_packet_rate, in case it needs some time to start flowing
        wait_time_s = 1
        if dp.read_attribute("rx_packet_rate").value == 0:
            sleep(wait_time_s)
        rx_packet_rate = dp.read_attribute("rx_packet_rate").value
        los_seconds = dp.read_attribute("los_seconds").value
        link_failure = dp.read_attribute("link_failure").value
        packet_loss_count = dp.read_attribute("packet_loss_count").value
        packet_loss = dp.read_attribute("packet_loss").value
        packet_error_count = dp.read_attribute("packet_error_count").value
        packet_error = dp.read_attribute("packet_error").value

        print(
            f"{device}:\n\tpacket_rate = {packet_rate}\n\trx_packet_rate = {rx_packet_rate}\n\tlos_seconds = {los_seconds}\n\tlink_failure = {link_failure}\n\tpacket_loss_count = {packet_loss_count}\n\tpacket_loss = {packet_loss}\n\tpacket_error_count = {packet_error_count}\n\tpacket_error = {packet_error}\n"
        )

### Post-VCC Histogram

In [ ]:
# Each board has 4 Post-VCC devices e_post_vcc_[0-3]
for vcc in vccs:
    for fsp in fsps:
        device = f"talondx-00{fsp}/histogram/e_post_vcc_{vcc-1}"
        timeout = 3000  # ms
        client = HistogramClient(device, timeout)

        # data is an array of [pol_x_data, pol_y_data], or None if success is false.
        success, data = client.capture()
        print(f"{device} histogram capture successful: {success}")

        if success:
            plot_histogram(device, data)

### Post-Resampler Delay Tracker (RDT) Histogram

In [ ]:
# Each board has 4 Post-RDT devices e_post_rdt_[0-3]
for vcc in vccs:
    for fsp in fsps:
        device = f"talondx-00{fsp}/histogram/e_post_rdt_{vcc-1}"
        timeout = 3000  # ms
        client = HistogramClient(device, timeout)

        # data is an array of [pol_x_data, pol_y_data], or None if success is false.
        success, data = client.capture()
        print(f"{device} histogram capture successful: {success}")

        if success:
            plot_histogram(device, data)

### Post-16K Channelizer Histogram

In [ ]:
# Each board has 4 Post-ch16k devices e_post_ch16k_[0-3]


for vcc in vccs:
    for fsp in fsps:
        device = f"talondx-00{fsp}/histogram/e_post_ch16k_{vcc-1}"
        timeout = 3000  # ms
        client = HistogramClient(device, timeout)
        channel = 7440
        client.set_channel(channel)

        # data is an array of [pol_x_data, pol_y_data], or None if success is false.
        success, data = client.capture()
        print(f"{device} histogram capture successful: {success}")

        if success:
            plot_histogram(device, data)

### DDR4 Corner Turner (DCT)

In [ ]:
for fsp in fsps:
    device = f"talondx-00{fsp}/dct/dct"
    dp = DeviceProxy(device)

    cell_channels = dp.read_attribute("cell_channels").value
    cell_size = dp.read_attribute("cell_size").value
    cell_samples = dp.read_attribute("cell_samples").value
    num_receptors = dp.read_attribute("num_receptors").value
    read_threshold = dp.read_attribute("read_threshold").value
    start_read_timestamp = dp.read_attribute("start_read_timestamp").value

    dp.write_attribute("snapshot_timestamps", True)

    read_timestamp = dp.read_attribute("read_timestamp").value

    first_write_timestamps = dp.read_attribute("first_write_timestamp").value

    write_timestamps = []
    write_timestamps = dp.read_attribute("write_timestamps").value

    antenna_status = dp.read_attribute("antenna_status").value
    antenna_status_str = ""
    for x in antenna_status:
        antenna_status_str = str(int(x)) + antenna_status_str
    under_run_status = dp.read_attribute("under_run_status").value
    under_run_status_str = ""
    for x in under_run_status:
        under_run_status_str = str(int(x)) + under_run_status_str

    print(f"Device: {device}")
    print(f"Cell channels = {cell_channels}")
    print(f"Cell size = {cell_size}")
    print(f"Cell samples = {cell_samples}")
    print(f"Number of receptors = {num_receptors}")
    print(f"Read threshold = {read_threshold}")
    print(f"Start read timestamp = {start_read_timestamp}")
    print(f"First write timestamps = {first_write_timestamps}")
    print(f"Read timestamps = {read_timestamp}")
    print(f"Write timestamps = {write_timestamps}")
    print(f"Antenna status = {antenna_status_str}")
    print(f"under run status = {under_run_status_str}")

### 100gbe Ethernet 

In [ ]:
from notebook_tools.signal_chain_checks import Eth100gClient

eth100g = Eth100gClient(vccs, fsps)
eth100g.clear_counters()
sleep(3)
tx_table, rx_table = eth100g.read_counters()
print(rx_table)
print(tx_table)